# Get SCOS Info

In [ ]:
import numpy as np
import pandas as pd
import pickle
from tqdm.notebook import tqdm,trange
from pprint import pprint
import pdb

In [ ]:
fpath = '/mnt/esa-ocai/ioa/karel'
fpath_remote = '../ioa/karel'

## GAIA

In [ ]:
gaia_parse = pickle.load(open('gaia_parsers.pkl','rb'))

In [ ]:
def get_single_parent(parents):
    p_list = ['aocs','stracker','gyro','fss','cps']
    for p in p_list:
        if len(parents)==1:
            return parents[0] if parents[0]!='stracker' else 'str'
        if p in val:
            parents.remove(p)
            return get_single_parent(parents)
    
    p_join = ''
    for p in parents:
        p_join += p
        
    if 'fss' in p_join:
        return 'fss'
    if 'cps' in p_join:
        return 'cps'
    
    return p_join

### ReportingData

In [ ]:
colnames_pcf = ['PCF_NAME', 'PCF_DESCR', 'PCF_PID', 'PCF_UNIT', 'PCF_PTC', 
                'PCF_PFC', 'PCF_WIDTH', 'PCF_VALID', 'PCF_RELATED', 'PCF_CATEG', 
                'PCF_NATUR', 'PCF_CURTX', 'PCF_INTER', 'PCF_USCON', 'PCF_DECIM',
                'PCF_PARVAL', 'PCF_SUBSYS', 'PCF_VALPAR', 'PCF_SPTYPE', 'PCF_CORR',
                'PCF_OBTID', 'PCF_DARC', 'PCF_ENDIAN']

In [ ]:
fname_gaia = fpath + '/gaia/s2k_mib/pcf.dat'
data_report_gaia = pd.read_csv(fname_gaia, sep='\t', names=colnames_pcf, index_col=False, usecols=[0,1,2,3,16])
data_report_gaia

In [ ]:
data_report_gaia['PCF_UNIT'].unique()

In [ ]:
parse_rd = gaia_parse['ReportingData']
parse_rd

In [ ]:
data_report_gaia['parents'] = [[] for _ in range(len(data_report_gaia))]
parent_masks = {}

for parse,parent in tqdm(zip(parse_rd['parser'], parse_rd['parent']), total=len(parse_rd['parent'])):
    parse_str = [p.decode("utf-8").strip('\'') for p in parse]
    parent_str = parent.decode("utf-8")
    
    name_mask = data_report_gaia['PCF_NAME'].str.contains(parse_str[0], regex=True, na=False)
    descr_mask = data_report_gaia['PCF_DESCR'].str.contains(parse_str[1], regex=True, na=False)
    subsys_mask = data_report_gaia['PCF_SUBSYS'].str.contains(parse_str[2], regex=True, na=False)
    
    parent_masks[parent_str] = name_mask & descr_mask & subsys_mask
    
    for ind in data_report_gaia.loc[name_mask & descr_mask & subsys_mask].index:
        data_report_gaia.loc[ind,'parents'].append(parent_str)

In [ ]:
data_out = data_report_gaia.loc[data_report_gaia['parents'].map(len)>0].copy()
data_out

In [ ]:
data_out['parents'].map(len).value_counts()

In [ ]:
data_out['single_parent'] = ['' for _ in range(len(data_out))]

for i, val in data_out['parents'].items():
    data_out.loc[i,'single_parent'] = [get_single_parent(val)]

In [ ]:
data_out.single_parent.value_counts()

In [ ]:
data_out

In [ ]:
data_out.loc[data_out.single_parent.str.contains("fss.*fss",regex=True,na=False)]

In [ ]:
data_out.loc[data_out.single_parent.str.contains("cps.*cps",regex=True,na=False)]

Example issues

In [ ]:
data_report_gaia.loc[(parent_masks['cpsa'] & parent_masks['cpsb']), 'PCF_DESCR'].values

In [ ]:
descr_cpsa = data_report_gaia['PCF_DESCR'].str.contains(".*CPS.*A.*", regex=True, na=False)
subsys_cpsa = data_report_gaia['PCF_SUBSYS'].str.contains('CPS', regex=True, na=False)

data_report_gaia.loc[descr_cpsa & subsys_cpsa]

### Activity

In [ ]:
colnames_ccf = ['CCF_CNAME', 'CCF_DESCR', 'CCF_DESCR2', 'CCF_CTYPE', 'CCF_CRITICAL', 
                'CCF_PKTID', 'CCF_TYPE', 'CCF_STYPE', 'CCF_APID', 'CCF_NPARS', 
                'CCF_PLAN', 'CCF_EXEC', 'CCF_ILSCOPE', 'CCF_ILSTAGE', 'CCF_SUBSYS',
                'CCF_HIPRI', 'CCF_MAPID', 'CCF_DEFSET', 'CCF_RAPID', 'CCF_ACK',
                'CCF_SUBSCHEDID']

In [ ]:
fname_gaia = fpath + '/gaia/s2k_mib/ccf.dat'
data_activity_gaia = pd.read_csv(fname_gaia, sep='\t', names=colnames_ccf, index_col=False, usecols=[0,1,2,6,7,8,14,20])
data_activity_gaia

In [ ]:
parse_act = gaia_parse['Activity']
parse_act

In [ ]:
data_activity_gaia['parents'] = [[] for _ in range(len(data_activity_gaia))]
parent_masks = {}

for parse,parent in tqdm(zip(parse_act['parser'], parse_act['parent']), total=len(parse_act['parent'])):
    parse_str = [p.decode("utf-8").strip('\'') for p in parse]
    parent_str = parent.decode("utf-8")
    
    name_mask = data_activity_gaia['CCF_CNAME'].str.contains(parse_str[0], regex=True, na=False)
    descr_mask = data_activity_gaia['CCF_DESCR'].str.contains(parse_str[1], regex=True, na=False)
    type_mask = ~data_activity_gaia['CCF_TYPE'].isnull()
    stype_mask = ~data_activity_gaia['CCF_STYPE'].isnull()
    apid_mask = ~data_activity_gaia['CCF_APID'].isnull()
    
    parent_masks[parent_str] = name_mask & descr_mask & type_mask & stype_mask & apid_mask
    
    for ind in data_activity_gaia.loc[name_mask & descr_mask & type_mask & stype_mask & apid_mask].index:
        data_activity_gaia.loc[ind,'parents'].append(parent_str)

In [ ]:
data_out_act = data_activity_gaia.loc[data_activity_gaia['parents'].map(len)>0].copy()
data_out_act

In [ ]:
data_out_act['parents'].map(len).value_counts()

In [ ]:
data_out_act['single_parent'] = ['' for _ in range(len(data_out_act))]

for i, val in data_out_act['parents'].items():
    data_out_act.loc[i,'single_parent'] = get_single_parent(val)

In [ ]:
data_out_act.single_parent.value_counts()

In [ ]:
data_out_act

### Event

In [ ]:
colnames_pid = ['PID_TYPE', 'PID_STYPE', 'PID_APID', 'PID_PI1_VAL', 'PID_PI2_VAL',
                'PID_SPID', 'PID_DESCR', 'PID_UNIT', 'PID_TPSD', 'PID_DFHSIZE',
                'PID_TIME', 'PID_INTER', 'PID_VALID', 'PID_CHECK', 'PID_EVENT',
                'PID_EVID']

In [ ]:
fname_gaia = fpath + '/gaia/s2k_mib/pid.dat'
data_event_gaia = pd.read_csv(fname_gaia, sep='\t', names=colnames_pid, index_col=False, usecols=[0,1,2,6,7])
data_event_gaia

In [ ]:
parse_ev = gaia_parse['Event']
parse_ev

In [ ]:
data_event_gaia['parents'] = [[] for _ in range(len(data_event_gaia))]
parent_masks = {}

for parse,parent in tqdm(zip(parse_ev['parser'], parse_ev['parent']), total=len(parse_ev['parent'])):
    parse_str = [p.decode("utf-8").strip('\'') for p in parse]
    parent_str = parent.decode("utf-8")

    type_mask = ~data_event_gaia['PID_TYPE'].isnull()
    stype_mask = ~data_event_gaia['PID_STYPE'].isnull()
    apid_mask = ~data_event_gaia['PID_APID'].isnull()
    descr_mask = data_event_gaia['PID_DESCR'].str.contains(parse_str[3], regex=True, na=False)
    
    parent_masks[parent_str] = type_mask & stype_mask & apid_mask & descr_mask
    
    for ind in data_event_gaia.loc[type_mask & stype_mask & apid_mask & descr_mask].index:
        data_event_gaia.loc[ind,'parents'].append(parent_str)

In [ ]:
data_out_ev = data_event_gaia.loc[data_event_gaia['parents'].map(len)>0].copy()
data_out_ev

In [ ]:
data_out_ev['parents'].map(len).value_counts()

In [ ]:
data_out_ev['single_parent'] = ['' for _ in range(len(data_out_ev))]

for i, val in data_out_ev['parents'].items():
    data_out_ev.loc[i,'single_parent'] = get_single_parent(val)

In [ ]:
data_out_ev.single_parent.value_counts()

In [ ]:
data_out_ev

## Cluster

In [ ]:
colnames_clu = ['PCF_NAME', 'PCF_DESCR', 'PCF_UNIT', 'PCF_VALID', 'PCF_CATEG',
                'PCF_WIDTH', 'PCF_NATUR', 'PCF_CODIN', 'PCF_CURTX', 'PCF_INTER',
                'PCF_USCON', 'PCF_CTSPE', 'PCF_CTREL', 'PCF_CTDIS', 'PCF_CTPRO',
                'PCF_DECIM', 'PCF_PARVAL', 'extra1']

In [ ]:
fname_clu = fpath + '/cluster/mib/clu2/PCF.dat'
data_report_cluster = pd.read_csv(fname_clu, names=colnames_clu ,index_col=False)
data_report_cluster

In [ ]:
data_report_cluster['PCF_UNIT'].unique()

In [ ]:
data_report_cluster.loc[1993:2000,:]